In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-6_o6zukd/unsloth_8026c3d69f1742b584d7562f3b750dfd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-6_o6zukd/unsloth_8026c3d69f1742b584d7562f3b750dfd
  Resolved https://github.com/unslothai/unsloth.git to commit b0781339f035c72b3028d846eb2261e8115cd375
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
  Created wheel for u

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Unsloth: You passed in `unsloth/Phi-3-mini-4k-instruct` and `load_in_4bit = True`.
We shall load `unsloth/Phi-3-mini-4k-instruct-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
functional_tokens = [f"<nexa_{i}>" for i in range(16)] + ["<nexa_end>"]  #Adding 15 nexa tokens

In [4]:
num_added_toks = tokenizer.add_tokens(functional_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(32028, 3072)

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


In [6]:
prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [7]:
from datasets import Dataset, DatasetDict
import pandas as pd

# Simulated dataset
data = {
    "instruction": [
        "Call the function to increase the volume.",
        "Call the function to decrease the volume.",
        "Retrieve the current weather report.",
        "Send an email to the user.",
        "Schedule a meeting.",
        "Fetch the latest news articles.",
        "Post a tweet on behalf of the user.",
        "Activate the security system.",
        "Deactivate the security system.",
        "Adjust the thermostat to a warmer setting.",
        "Adjust the thermostat to a cooler setting.",
        "Play the next song in the playlist.",
        "Pause the current song.",
        "Switch the lights to night mode.",
        "Switch the lights to day mode."
    ],
    "input": [
        "User increases volume by 5 units.",
        "User decreases volume by 3 units.",
        "User asks for the weather in New York.",
        "User wants to email John Doe.",
        "User needs to meet with the finance team tomorrow.",
        "User is interested in today's technology news.",
        "User wants to tweet about their new product launch.",
        "User says 'activate the alarm'.",
        "User says 'deactivate the alarm'.",
        "User feels cold and wants it warmer.",
        "User feels hot and wants it cooler.",
        "User wants to listen to more music.",
        "User needs to pause the music for a call.",
        "User is preparing for bed and needs softer lighting.",
        "User starts their day and wants brighter lights."
    ],
    "output": [
        "<nexa_1>();<nexa_end>",
        "<nexa_2>();<nexa_end>",
        "<nexa_3>();<nexa_end>",
        "<nexa_4>();<nexa_end>",
        "<nexa_5>();<nexa_end>",
        "<nexa_6>();<nexa_end>",
        "<nexa_7>();<nexa_end>",
        "<nexa_8>();<nexa_end>",
        "<nexa_9>();<nexa_end>",
        "<nexa_10>();<nexa_end>",
        "<nexa_11>();<nexa_end>",
        "<nexa_12>();<nexa_end>",
        "<nexa_13>();<nexa_end>",
        "<nexa_14>();<nexa_end>",
        "<nexa_15>();<nexa_end>"
    ]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert DataFrame to Hugging Face dataset
hf_dataset = Dataset.from_pandas(df)

# Define the function to format data
def formatting_prompts_func(examples):
    prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

    texts = [prompt_template.format(instr, inp, out)
             for instr, inp, out in zip(examples['instruction'], examples['input'], examples['output'])]
    return {"text": texts}

# Apply the function on the dataset
formatted_dataset = hf_dataset.map(formatting_prompts_func, batched=True)

# Display the formatted texts for verification
for example in formatted_dataset:
    print(example['text'])
    print("\n---\n")


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Call the function to increase the volume.

### Input:
User increases volume by 5 units.

### Response:
<nexa_1>();<nexa_end>

---

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Call the function to decrease the volume.

### Input:
User decreases volume by 3 units.

### Response:
<nexa_2>();<nexa_end>

---

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Retrieve the current weather report.

### Input:
User asks for the weather in New York.

### Response:
<nexa_3>();<nexa_end>

---

Below is an instruction that describes a task, paired with an input that provid

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [9]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.523 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,117,248


Step,Training Loss
1,2.589500
2,2.638300
3,2.541900
4,2.419300
5,2.166300
6,1.985900
7,1.754900
8,1.694300
9,1.557000
10,1.545000


In [11]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

343.1764 seconds used for training.
5.72 minutes used for training.
Peak reserved memory = 2.607 GB.
Peak reserved memory for training = 0.084 GB.
Peak reserved memory % of max memory = 17.677 %.
Peak reserved memory for training % of max memory = 0.57 %.


In [12]:

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Fetch the latest news articles.", # instruction
        "User is interested in today's technology news.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nFetch the latest news articles.\n\n### Input:\nUser is interested in today's technology news.\n\n### Response:\n<nexa_6>();<nexa_end>\n<nexa>Fetch the latest technology news articles for today.</nexa>\n<nexa_end>\n\n### Solution 1:\n<nexa_6>();<nexa_end>\n<nexa>Retr"]